In [ ]:
!pip install ultralytics

import os
import shutil
import json
import random
from IPython.display import clear_output, Image
import glob
import ast
import cv2
import pandas as pd

clear_output()

In [2]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.7/112.6 GB disk)


In [3]:
!nvidia-smi

Sat Aug 16 10:25:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Connet to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Constants
IMAGE_WIDTH: int = 1000
IMAGE_HEIGHT: int = 800
dataset_dir: str = '/content/cataract_dataset_segmentation'
zip_file_path: str = '/content/drive/MyDrive/Datasets/insegcat-1-v2.zip'

## Preproccess Dataset

In [6]:
class PreprocessData():
    def __init__(self, dataset_dir, zip_file_path):
        self.dataset_dir = dataset_dir
        self.zip_file_path = zip_file_path
        self.images_dir = ""
        self.labels_dir = ""

    # Dataset directory and zip file path to be extracted
    def extract_dataset(self):
        if not os.path.exists(self.zip_file_path):
            print("File not found in Google Drive path.")
        else:
            os.makedirs(self.dataset_dir, exist_ok=True)
            !unzip -q "{self.zip_file_path}" -d "{self.dataset_dir}"
            print("File opened successfully")

    # move images in the dataset_dir/images
    def organize_images(self):
        images_dir = os.path.join(self.dataset_dir, 'images')
        self.images_dir = images_dir
        os.makedirs(images_dir, exist_ok=True)

        for filename in os.listdir(self.dataset_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                shutil.move(os.path.join(self.dataset_dir, filename), os.path.join(images_dir, filename))
        print("All images moved into 'images' folder.")

    # COCO JSON preprocess
    def map_categories(self, json_file: str):
        json_file = os.path.join(self.dataset_dir, json_file)

        with open(json_file, 'r') as f:
            coco_data = json.load(f)

        categories_coco = coco_data['categories']
        cat_ids = [cat['id'] for cat in categories_coco]
        cat_names = [cat['name'] for cat in categories_coco]
        self.cat_names = cat_names
        cat_id_to_yolo = {cat_id: idx for idx, cat_id in enumerate(cat_ids)}

        print("Categories mapping:")
        for cat_id, name in zip(cat_ids, cat_names):
            print(f"COCO id {cat_id} → YOLO id {cat_id_to_yolo[cat_id]} → name '{name}'")

        return cat_id_to_yolo

    # Convert coco to yolo format and for object detection
    def convert_coco_to_yolo(self, coco_json_path, output, cat_id_to_yolo_map):
        self.labels_dir = output

        if os.path.exists(self.labels_dir):
            shutil.rmtree(self.labels_dir)
            print(f"Deleted existing folder: {self.labels_dir}")
        os.makedirs(self.labels_dir)

        with open(coco_json_path, 'r') as f:
            coco_data = json.load(f)

        images_info = {
            img['id']: {'width': img['width'], 'height': img['height'], 'file_name': img['file_name']}
            for img in coco_data['images']
        }

        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = cat_id_to_yolo_map[annotation['category_id']]

            # mapping = {
            #     "i1": "slit knife",
            #     "i2": "angled incision knife",
            #     "i3": "katena forceps",
            #     "i7": "27 gauge cannula",
            #     "i8": "capsulorhexis forceps",
            #     "i9": "cannula",
            #     "i10": "phaco tip",
            #     "i11": "spatula",
            #     "i12": "I/A handpiece",
            #     "i16": "cartridge",
            #     "i20": "eye retractors"
            # }

            # category_id = mapping[category_id]

            if image_id not in images_info:
                continue

            img_info = images_info[image_id]
            w, h = img_info['width'], img_info['height']
            img_file_name = img_info['file_name']

            xmin, ymin, bbox_w, bbox_h = annotation['bbox']
            x_center = (xmin + bbox_w / 2) / w
            y_center = (ymin + bbox_h / 2) / h
            norm_w = bbox_w / w
            norm_h = bbox_h / h

            yolo_line = f"{category_id} {x_center} {y_center} {norm_w} {norm_h}\n"

            txt_file_path = os.path.join(self.labels_dir, os.path.splitext(img_file_name)[0] + '.txt')
            with open(txt_file_path, 'a') as f:
                f.write(yolo_line)

        print("COCO to YOLO conversion completed.")

    # Counts the number of files in a given directory using os.scandir().
    def count_files_os_scandir(self, directory_path):
        count = 0
        try:
            for entry in os.scandir(directory_path):
                if entry.is_file():
                    count += 1
            return count
        except FileNotFoundError:
            print(f"Error: Directory not found at '{directory_path}'")
            return -1
        except Exception as e:
            print(f"An error occurred: {e}")
            return -1

    # Splitting data to train, test and validation sets
    def spilt_dataset(self):
        random.seed(42)
        all_images = [f for f in os.listdir(self.images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(all_images)

        total_files = len(all_images)
        train_split_count = int(total_files * 0.8)
        val_split_count = int(total_files * 0.1)

        train_files = all_images[:train_split_count]
        val_files = all_images[train_split_count:train_split_count + val_split_count]
        test_files = all_images[train_split_count + val_split_count:]

        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(self.dataset_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(self.dataset_dir, 'labels', split), exist_ok=True)

        print("Dataset split completed!")
        return train_files, val_files, test_files

    # Moving images to the related folders
    def move_files(self, file_list, split):
        for file_name in file_list:
            # image
            src_img = os.path.join(self.images_dir, file_name)
            dst_img = os.path.join(self.dataset_dir, 'images', split, file_name)
            if os.path.exists(src_img):
                shutil.move(src_img, dst_img)

            # label
            label_name = os.path.splitext(file_name)[0] + '.txt'
            src_lbl = os.path.join(self.labels_dir, label_name)
            dst_lbl = os.path.join(self.dataset_dir, 'labels', split, label_name)
            if os.path.exists(src_lbl):
                shutil.move(src_lbl, dst_lbl)

        print(f"images moved to {split} folder")

    # Create data.yaml file
    def create_yaml_file(self):
        yaml_content = f"""
        path: {self.dataset_dir}

        train: images/train
        val: images/val
        test: images/test

        nc: {len(self.cat_names)}
        names: {self.cat_names}
        """
        with open(os.path.join(self.dataset_dir, 'data.yaml'), 'w') as f:
            f.write(yaml_content)

        print("data.yaml created successfully!")

In [7]:
preprocess = PreprocessData(dataset_dir, zip_file_path)
preprocess.extract_dataset()

File opened successfully


In [8]:
preprocess.organize_images()

All images moved into 'images' folder.


In [9]:
json_file = os.path.join(dataset_dir, 'coco-annotations.json')
cat_id_to_yolo = preprocess.map_categories(json_file)

labels_dir = os.path.join(dataset_dir, 'labels')
preprocess.convert_coco_to_yolo(json_file, labels_dir, cat_id_to_yolo)

Categories mapping:
COCO id 1 → YOLO id 0 → name 'i1'
COCO id 2 → YOLO id 1 → name 'i2'
COCO id 3 → YOLO id 2 → name 'i3'
COCO id 4 → YOLO id 3 → name 'i7'
COCO id 5 → YOLO id 4 → name 'i8'
COCO id 6 → YOLO id 5 → name 'i9'
COCO id 7 → YOLO id 6 → name 'i10'
COCO id 8 → YOLO id 7 → name 'i11'
COCO id 9 → YOLO id 8 → name 'i12'
COCO id 10 → YOLO id 9 → name 'i16'
COCO id 13 → YOLO id 10 → name 'i20'
COCO to YOLO conversion completed.


In [10]:
directory = [dataset_dir + "/images", labels_dir]
for d in directory:
  num_files = preprocess.count_files_os_scandir(d)
  if num_files != -1:
      print(f"Number of files in '{d}': {num_files}")

Number of files in '/content/cataract_dataset_segmentation/images': 843
Number of files in '/content/cataract_dataset_segmentation/labels': 843


In [11]:
train_files, val_files, test_files = preprocess.spilt_dataset()
preprocess.move_files(train_files, 'train')
preprocess.move_files(val_files, 'val')
preprocess.move_files(test_files, 'test')

Dataset split completed!
images moved to train folder
images moved to val folder
images moved to test folder


In [12]:
preprocess.create_yaml_file()

data.yaml created successfully!


In [13]:
# Removing i20 class
labels_root = os.path.join(dataset_dir, "labels")
images_root = os.path.join(dataset_dir, "images")
data_yaml_path = os.path.join(dataset_dir, "data.yaml")

remove_class_name = "i20"

def read_names_from_data_yaml(yaml_path):
    if not os.path.exists(yaml_path):
        raise FileNotFoundError(f"{yaml_path} not found. Please ensure data.yaml exists.")
    with open(yaml_path, "r", encoding="utf-8") as f:
        text = f.read()
    for line in text.splitlines():
        if line.strip().startswith("names:"):
            names_str = line.split("names:",1)[1].strip()
            try:
                names = ast.literal_eval(names_str)
                return list(names)
            except Exception:
                after = text.split("names:",1)[1].strip()
                try:
                    names = ast.literal_eval(after)
                    return list(names)
                except Exception as e:
                    raise RuntimeError("Couldn't parse names from data.yaml. Please ensure names: is a python list on one line.") from e
    raise RuntimeError("names: not found in data.yaml")

names = read_names_from_data_yaml(data_yaml_path)
if remove_class_name not in names:
    raise ValueError(f"Class '{remove_class_name}' not found in data.yaml names. Current names: {names}")
remove_id = names.index(remove_class_name)
print(f"Will remove class '{remove_class_name}' with current id = {remove_id}")

def remove_class_and_remap_labels(labels_root, remove_id):
    splits = ["train", "val", "test"]
    for split in splits:
        lbl_dir = os.path.join(labels_root, split)
        if not os.path.isdir(lbl_dir):
            continue
        for lbl_path in glob.glob(os.path.join(lbl_dir, "*.txt")):
            with open(lbl_path, "r") as f:
                lines = [ln.strip() for ln in f.readlines() if ln.strip()]
            new_lines = []
            for ln in lines:
                parts = ln.split()
                cls = int(float(parts[0]))
                coords = parts[1:]
                if cls == remove_id:
                    continue

                if cls > remove_id:
                    cls = cls - 1
                new_lines.append(" ".join([str(cls)] + coords) + "\n")
            with open(lbl_path, "w") as f:
                f.writelines(new_lines)
    print("Labels processed: remove + remap done (images kept).")

remove_class_and_remap_labels(labels_root, remove_id)

names.pop(remove_id)
nc = len(names)

yaml_content = f"""
path: {dataset_dir}

train: images/train
val: images/val
test: images/test

nc: {nc}
names: {names}
"""
with open(data_yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)

print(f"data.yaml updated. nc={nc}. New names: {names}")

Will remove class 'i20' with current id = 10
Labels processed: remove + remap done (images kept).
data.yaml updated. nc=10. New names: ['i1', 'i2', 'i3', 'i7', 'i8', 'i9', 'i10', 'i11', 'i12', 'i16']


## Models

1. Baseline Model Training
    - Trained an initial YOLO-based detection model with default/random augmentation parameters.
    - This served as the baseline model for comparison.

2. Hyperparameter Optimization with Optuna
    - Used Optuna to search for the best augmentation hyperparameters (e.g., fliplr, flipud, mosaic, scale, translate, hsv_h, hsv_s, hsv_v).
    - Identified the optimal augmentation configuration that maximized detection performance on the validation dataset.

3. Model Retraining with Optimized Augmentations
    - Retrained the model using the best augmentation hyperparameters found by Optuna.
    - Compared the new model’s metrics (Precision, Recall, mAP50, mAP50-95) with the baseline model.
    - Results showed improvements in overall generalization, particularly in mAP50-95, while keeping performance close in other metrics.

4. Evaluation on Video Data
    - Applied the trained model to real surgical video footage.
    - Generated detection outputs frame-by-frame to evaluate how well the model generalizes to unseen, real-world surgical tools (including instruments not present in the training dataset).

In [34]:
!yolo detect train \
  model=yolov8n.pt \
  data=/content/cataract_dataset_segmentation/data.yaml \
  epochs=100 \
  batch=16 \
  workers=2 \
  project=/content/drive/MyDrive/yolo_runs \
  name=baseline_oph \
  plots=True \
  fliplr=0.0 flipud=0.0 mosaic=0.05 mixup=0.0 copy_paste=0.0 \
  degrees=10.0 translate=0.1 scale=0.2 shear=0.0 perspective=0.0 \
  hsv_h=0.015 hsv_s=0.50 hsv_v=0.30


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/cataract_dataset_segmentation/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.05, multi_scale=False, name=baseline_oph, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plo

In [35]:
!yolo val model=/content/drive/MyDrive/yolo_runs/baseline_oph/weights/best.pt data=/content/cataract_dataset_segmentation/data.yaml \
  plots=True save_json=True

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1621.4±567.7 MB/s, size: 491.7 KB)
val: Scanning /content/cataract_dataset_segmentation/labels/val.cache... 84 images, 0 backgrounds, 0 corrupt: 100% 84/84 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:01<00:00,  3.81it/s]
                   all         84        109      0.958      0.946       0.97      0.835
                    i1          8          8      0.965          1      0.995      0.898
                    i2         13         13          1      0.954      0.995      0.889
                    i3         12         12      0.981          1      0.995      0.797
                    i7         12         12      0.907      0.819      0.879       0.83
                    i8         17         17 

In [41]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.7 MB/s eta 0:00:00


In [ ]:
import optuna
from ultralytics import YOLO

DATA = "/content/cataract_dataset_segmentation/data.yaml"
MODEL = "yolov8n.pt"
PROJECT = "/content/drive/MyDrive/yolo_runs"
NAME = "optuna_search"

def objective(trial):
    hyp = {
        "fliplr": trial.suggest_float("fliplr", 0.0, 0.5),
        "flipud": trial.suggest_float("flipud", 0.0, 0.5),
        "mosaic": trial.suggest_float("mosaic", 0.0, 1.0),
        "scale": trial.suggest_float("scale", 0.0, 0.5),
        "translate": trial.suggest_float("translate", 0.0, 0.3),
        "hsv_h": trial.suggest_float("hsv_h", 0.0, 0.1),
        "hsv_s": trial.suggest_float("hsv_s", 0.0, 0.9),
        "hsv_v": trial.suggest_float("hsv_v", 0.0, 0.9),
    }

    model = YOLO(MODEL)
    results = model.train(
        data=DATA,
        epochs=10,
        batch=16,
        workers=2,
        project=PROJECT,
        name=NAME,
        imgsz=640,
        **hyp
    )

    map50_95 = results.results_dict["metrics/mAP50-95(B)"]
    return map50_95

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:", study.best_trial.params)


[I 2025-08-16 11:41:15,721] A new study created in memory with name: no-name-34fc2061-295a-402a-b649-d0abe4d4fbf6


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/cataract_dataset_segmentation/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.07427008490411208, flipud=0.11991518285373581, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0320968439823399, hsv_s=0.6212191100929898, hsv_v=0.39646009803101756, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.8086698885753737, multi_scale=False, name=optuna_search2, nbs=64, nms=False, opset=N

train: Scanning /content/cataract_dataset_segmentation/labels/train.cache... 674 images, 0 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1162.8±1362.7 MB/s, size: 509.0 KB)


...

                   all         84        109      0.772      0.882      0.871      0.732

10 epochs completed in 0.046 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_runs/optuna_search21/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/yolo_runs/optuna_search21/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/yolo_runs/optuna_search21/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.41it/s]


                   all         84        109      0.773      0.882      0.873      0.735
                    i1          8          8      0.797          1      0.928      0.866
                    i2         13         13      0.841          1      0.974      0.841
                    i3         12         12      0.911      0.859      0.945      0.743
                    i7         12         12      0.523      0.733      0.608      0.566
                    i8         17         17          1      0.865      0.942      0.817
                    i9         10         10      0.604      0.915      0.866      0.776
                   i10          4          4       0.68          1      0.895      0.765
                   i11         13         13      0.744      0.769      0.779      0.545
                   i12         10         10      0.718        0.7      0.807      0.683
                   i16         10         10      0.907      0.978      0.986      0.745
Speed: 0.1ms preproce

[I 2025-08-16 12:36:41,595] Trial 19 finished with value: 0.7346683038473644 and parameters: {'fliplr': 0.3432307750534994, 'flipud': 0.27881444385522536, 'mosaic': 0.5431911785692327, 'scale': 0.19529828991358972, 'translate': 0.06376875981370812, 'hsv_h': 0.04406957698335906, 'hsv_s': 0.5038666614825599, 'hsv_v': 0.4448012646385688}. Best is trial 8 with value: 0.8063821748536675.


Best trial: {'fliplr': 0.18254586508829868, 'flipud': 0.1026426415443632, 'mosaic': 0.5985582700563097, 'scale': 0.07854232818482026, 'translate': 0.0385847670214391, 'hsv_h': 0.08080319371127404, 'hsv_s': 0.7603012085960043, 'hsv_v': 0.23699064954831744}


In [46]:
!yolo detect train \
  model=yolov8n.pt \
  data=/content/cataract_dataset_segmentation/data.yaml \
  epochs=100 \
  batch=16 \
  workers=2 \
  project=/content/drive/MyDrive/yolo_runs \
  name=final_best_aug \
  fliplr=0.18 flipud=0.10 mosaic=0.60 \
  scale=0.08 translate=0.04 \
  hsv_h=0.08 hsv_s=0.76 hsv_v=0.24


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/cataract_dataset_segmentation/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.18, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.08, hsv_s=0.76, hsv_v=0.24, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=final_best_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

In [47]:
!yolo detect val model=/content/drive/MyDrive/yolo_runs/final_best_aug/weights/best.pt data=/content/cataract_dataset_segmentation/data.yaml

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1974.5±1343.6 MB/s, size: 714.4 KB)
val: Scanning /content/cataract_dataset_segmentation/labels/val.cache... 84 images, 0 backgrounds, 0 corrupt: 100% 84/84 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:01<00:00,  3.74it/s]
                   all         84        109      0.955      0.919      0.951      0.858
                    i1          8          8      0.974          1      0.995      0.955
                    i2         13         13      0.996          1      0.995      0.878
                    i3         12         12       0.99          1      0.995      0.846
                    i7         12         12      0.966       0.75      0.872       0.81
                    i8         17         17

In [48]:
!yolo detect val \
  model=/content/drive/MyDrive/yolo_runs/final_best_aug/weights/best.pt \
  data=/content/cataract_dataset_segmentation/data.yaml \
  split=test

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 50.0±47.7 MB/s, size: 504.0 KB)
val: Scanning /content/cataract_dataset_segmentation/labels/test... 85 images, 0 backgrounds, 0 corrupt: 100% 85/85 [00:00<00:00, 184.31it/s]
val: New cache created: /content/cataract_dataset_segmentation/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:01<00:00,  3.70it/s]
                   all         85        109      0.948      0.924      0.975      0.899
                    i1          9          9       0.99          1      0.995      0.959
                    i2         13         13      0.911          1      0.979      0.872
                    i3         12         12      0.852          1      0.984      0.914
                    i7          8          8      0.

In [ ]:
import pandas as pd

baseline_model_path = "/content/drive/MyDrive/yolo_runs/baseline_oph/weights/best.pt"
optuna_model_path   = "/content/drive/MyDrive/yolo_runs/final_best_aug/weights/best.pt"

data_yaml = "/content/cataract_dataset_segmentation/data.yaml"

baseline = YOLO(baseline_model_path)
optuna_model = YOLO(optuna_model_path)

baseline_metrics = baseline.val(data=data_yaml, split="test")
optuna_metrics   = optuna_model.val(data=data_yaml, split="test")

results = pd.DataFrame({
    "Metric": ["Precision", "Recall", "mAP50", "mAP50-95"],
    "Baseline": [
        baseline_metrics.results_dict["metrics/precision(B)"],
        baseline_metrics.results_dict["metrics/recall(B)"],
        baseline_metrics.results_dict["metrics/mAP50(B)"],
        baseline_metrics.results_dict["metrics/mAP50-95(B)"]
    ],
    "Optuna-Aug": [
        optuna_metrics.results_dict["metrics/precision(B)"],
        optuna_metrics.results_dict["metrics/recall(B)"],
        optuna_metrics.results_dict["metrics/mAP50(B)"],
        optuna_metrics.results_dict["metrics/mAP50-95(B)"]
    ]
})

print(results)

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2327.5±1399.7 MB/s, size: 501.6 KB)


val: Scanning /content/cataract_dataset_segmentation/labels/test.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.98it/s]


                   all         85        109      0.951      0.941      0.975      0.883
                    i1          9          9      0.985          1      0.995      0.959
                    i2         13         13      0.989          1      0.995      0.883
                    i3         12         12      0.918          1      0.957      0.854
                    i7          8          8      0.745      0.731      0.864      0.744
                    i8          8          8      0.981          1      0.995      0.891
                    i9         11         11      0.931      0.909      0.988      0.892
                   i10         10         10      0.973          1      0.995      0.897
                   i11         14         14          1      0.913      0.972      0.888
                   i12         12         12          1      0.859      0.995      0.938
                   i16         12         12      0.985          1      0.995      0.887
Speed: 6.2ms preproce

val: Scanning /content/cataract_dataset_segmentation/labels/test.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.46it/s]


                   all         85        109      0.948      0.924      0.975      0.899
                    i1          9          9       0.99          1      0.995      0.959
                    i2         13         13      0.911          1      0.979      0.872
                    i3         12         12      0.852          1      0.984      0.914
                    i7          8          8      0.962      0.625      0.928        0.8
                    i8          8          8      0.988          1      0.995      0.965
                    i9         11         11      0.834      0.912      0.973      0.876
                   i10         10         10      0.955        0.9      0.986      0.915
                   i11         14         14      0.997      0.929       0.99      0.891
                   i12         12         12          1      0.871      0.923      0.877
                   i16         12         12      0.993          1      0.995      0.917
Speed: 3.5ms preproce

In [49]:
!yolo detect predict \
  model=/content/drive/MyDrive/yolo_runs/final_best_aug/weights/best.pt \
  source=/content/drive/MyDrive/video/case_2000.mp4 \
  save=True \
  conf=0.25 \
  project=/content/drive/MyDrive/yolo_runs name=video_test

Streaming output truncated to the last 5000 lines.
video 1/1 (frame 37760/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 3 i7s, 7.5ms
video 1/1 (frame 37761/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.5ms
video 1/1 (frame 37762/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.8ms
video 1/1 (frame 37763/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.9ms
video 1/1 (frame 37764/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.7ms
video 1/1 (frame 37765/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.6ms
video 1/1 (frame 37766/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 7.4ms
video 1/1 (frame 37767/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 (no detections), 13.0ms
video 1/1 (frame 37768/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 8.9ms
video 1/1 (frame 37769/42756) /content/drive/MyDrive/video/case_2000.mp4: 384x640 1 i7, 8.

When evaluated on surgical video footage, the model performed not bad overall, correctly detecting and classifying most instruments in real time. Some occasional confusions were observed between visually similar tools, such as i7: 27-gauge cannula vs. i9: cannula, and in a few cases between i12: I/A handpiece and i16: cartridge. Nevertheless, the majority of instruments were recognized accurately, demonstrating the model’s strong potential for application in surgical video analysis.